# 1. Collecting (데이터 수집 및 클렌징)

In [ ]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')


# 파일 경로 지정
os.chdir('/content/drive/MyDrive/Project_Tesla')

In [ ]:
# 모듈 임포트 선언
import os
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re, unicodedata
from string import whitespace


# 네이버뉴스 1개 크롤링
def news(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    title_element = soup.select_one('h2#title_area')
    title = title_element.text if title_element else None
    date_element = soup.select_one('span.media_end_head_info_datestamp_time')
    date = date_element.get('data-date-time') if date_element else None
    content_element = soup.select_one('article#dic_area')
    content = content_element.text.strip() if content_element else None
    return {
        'title': title,
        'date': date,
        'content': content
    }


# 네이버뉴스 페이지+원하는 날짜 크롤링
def news_list(keyword, startdate, enddate):
    li = []
    h = {'User-Agent': '...',
         'Referer': '...',
         'cookie': '...'}
    for d in pd.date_range(startdate, enddate):
        str_d = d.strftime("%Y.%m.%d")
        page = 1
        print(str_d)
        while True:
            start = (page - 1) * 10 + 1
            print(page)
            URL = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query={0}&sort=2&photo=0&field=0&pd=3&ds={1}&de={2}&mynews=0&office_type=0&office_section_code=0&news_office_checked=&office_category=0&service_area=0&nso=so:r,p:from{3}to{4},a:all&start={5}".format(keyword, str_d, str_d, str_d.replace(".", ""), str_d.replace(".", ""), start)
            res = requests.get(URL, headers=h)
            soup = BeautifulSoup(res.text, "html.parser")
            if soup.select_one(".api_noresult_wrap"):
                break
            news_list = soup.select("ul.list_news li")
            for item in news_list:
                if len(item.select("div.info_group a")) == 2:
                    li.append(news(item.select("div.info_group a")[1]['href']))
            page = page + 1
    return pd.DataFrame(li, columns=['title', 'date', 'content'])


# 바이라인 제거
def clean_byline(text):
    # 바이라인
    pattern_email = re.compile(r'[-_0-9a-z]+@[-_0-9a-z]+(?:\.[0-9a-z]+)+', flags=re.IGNORECASE)
    pattern_url = re.compile(r'(?:https?:\/\/)?[-_0-9a-z]+(?:\.[-_0-9a-z]+)+', flags=re.IGNORECASE)
    pattern_others = re.compile(r'\.([^\.]*(?:기자|특파원|교수|작가|대표|논설|고문|주필|부문장|팀장|장관|원장|연구원|이사장|위원|실장|차장|부장|에세이|화백|사설|소장|단장|과장|기획자|큐레이터|저작권|평론가|©|©|ⓒ|\@|\/|=|▶|무단|전재|재배포|금지|\[|\]|\(\))[^\.]*)$')
    result = pattern_email.sub('', text)
    result = pattern_url.sub('', result)
    result = pattern_others.sub('.', result)

    # 본문 시작 전 꺽쇠로 쌓인 바이라인 제거
    pattern_bracket = re.compile(r'^((?:\[.+\])|(?:【.+】)|(?:<.+>)|(?:◆.+◆)\s)')
    result = pattern_bracket.sub('', result).strip()

    return result


# 크롤링할 데이터 (키워드, 시작날짜, 종료날짜)
result_df = news_list('테슬라', '2022.08.01', '2022.08.31')


# 크롤링 데이터, 데이터 프레임에 저장 및 필요없는 column 삭제
df = pd.DataFrame(result_df)
df['content'] = df['content'].fillna('').astype(str).map(clean_byline)


# 유니코드 문자 전처리 및 정규 표현식 사용
pattern_whitespace = re.compile(f'[{whitespace}]+')
df['content'] = df['content'].str.replace(pattern_whitespace, ' ').map(lambda x: unicodedata.normalize('NFC', x)).str.strip()


# 클렌징 데이터 csv 파일로 저장
df.to_csv('[2022-Aug]news_data_cleansing.csv', index=False, encoding='utf-8-sig')


# 2. Preprocessing (형태소 분석, 불용어 처리)

In [ ]:
!pip install konlpy

In [9]:
from konlpy.tag import Okt
from gensim import corpora


# csv 파일 불러오기
df = pd.read_csv('[2022-Aug]news_data_cleansing.csv')


# 형태소 분석기 초기화
okt = Okt()


# 텍스트 데이터를 리스트로 변환
documents = df['content'].tolist()


# 불용어 리스트 정의
stop_words = ["것", "수", "이", "그", "를", "를", "등", "과", "에", "가", '때', '의', '및']


# 각 문서를 형태소 분석 및 토큰화하고 불용어 제거
tokenized_documents = []
for document in documents:
    # 형태소 분석 수행 후 명사만 선택 (원하는 형태소 선택 가능)
    tokens = [word for word, pos in okt.pos(str(document)) if pos in ['Noun'] and word not in stop_words]
    tokenized_documents.append(tokens)


# 사전 (Dictionary) 생성
dictionary = corpora.Dictionary(tokenized_documents)

# 3. Analysis (토픽 모델링)

In [ ]:
from gensim.models import LdaModel, TfidfModel


# Tfidf 모델 생성
tfidf = TfidfModel(dictionary=dictionary)
corpus = [dictionary.doc2bow(tokens) for tokens in tokenized_documents]


# LDA 모델 생성
lda_model = LdaModel(corpus, num_topics=30, id2word=dictionary, passes=15)


# LDA 모델 출력
for idx, topic in lda_model.print_topics(-1):
    print(f"Topic #{idx}: {topic}")